In [146]:
import time # 로딩시간동안 대기하기 위해 time 모듈 사용
import pandas as pd # 데이터프레임 사용을 위해 pandas 모듈 사용
import requests # 서버에 접속해서 html 문서를 가져오는 역할(요청 -> 응답)
from bs4 import BeautifulSoup # html 문서를 해석하는 역할(파싱)
from selenium import webdriver # 크롤링에 사용하는 selenium
from selenium.webdriver.common.keys import Keys 
import openpyxl # excel 저장을 위해 openpyxl 모듈 사용

# 1. YES24

In [125]:
# 1. 예스24 크롤링
yes24 = 'http://www.yes24.com/Main/default.aspx'
request = requests.get(yes24) 
print(request) # <Response [200]> 정상적인 접속

<Response [200]>


In [155]:
# chromedriver로 페이지 열기
driver = webdriver.Chrome('D:/tests/chromedriver.exe') 
driver.get(yes24)
time.sleep(0.5)

c:\Users\user\AppData\Local\Programs\Python\Python37\lib\site-packages\ipykernel_launcher.py:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  


In [156]:
# 검색창에 파이썬 입력
search = driver.find_element('id', 'query')
search.send_keys('파이썬')
search.send_keys(Keys.RETURN)
time.sleep(0.5)

In [157]:
# 책 정보 크롤링
total_yes24 = []
next = True
page = 1
index = 0
while next :
# http://www.yes24.com/Product/Search?domain=ALL&query=%ED%8C%8C%EC%9D%B4%EC%8D%AC&page=1 ~ 5

    # 2 ~ 5페이지로 이동시키고, 5페이지일 경우 멈추도록
    if page == 5 : 
        next = False
    if 2 <= page <= 5 :
        url = 'http://www.yes24.com/Product/Search?domain=ALL&query=%ED%8C%8C%EC%9D%B4%EC%8D%AC&page={}'.format(page)
        driver.get(url)
        driver.implicitly_wait(10)
        time.sleep(0.2)
    print("page : ", page)

    # bs4로 문서 정보를 가져옴
    html = driver.page_source
    soup = BeautifulSoup(html)

    # 페이지 도서 리스트
    book_list = soup.find('ul', {'id' : 'yesSchList'})
    # 책 제목 태그 리스트
    book_name = book_list.select('.itemUnit > .item_info > .info_name > a.gd_name')
    # 가격 태그 리스트
    book_price = book_list.select('.itemUnit > .item_info > .info_price > strong.txt_num > em.yes_b')
    # 출판사 태그 리스트
    book_publish = book_list.select(".itemUnit > .item_info > .info_pubGrp > .authPub.info_pub")
    # 출판일자 태그 리스트
    book_date = book_list.select(".itemUnit > .item_info > .info_pubGrp > .authPub.info_date")

    print(len(book_name), len(book_price), len(book_publish), len(book_date))
    # book_name list의 갯수만큼 반복문을 돌려 2차원 리스트의 형태로 저장
    for i in range(len(book_name)) :
        # 책 제목과 가격이 빈 값이 아닐 경우
        if book_name[i].text.strip() and book_price[i].text.strip() :
            # 책제목, 가격, 출판사, 출판일자를 각각 저장
            books = [book_name[i].text.strip(), book_price[i].text.strip(), book_publish[i].text.strip(), book_date[i].text.strip()]
            total_yes24.append(books)
            index += 1
    page += 1
    time.sleep(0.2)

page :  1
24 24 24 24
page :  2
24 24 24 24
page :  3
24 24 24 24
page :  4
24 24 24 24
page :  5
24 24 24 24


In [158]:
# 이차원리스트를 데이터프레임으로 변환 및 컬럼 지정
total_yes24_columns = ['책제목', '가격', '출판사', '출판일자']
total_yes24 = pd.DataFrame(total_yes24)
total_yes24.columns = total_yes24_columns

In [159]:
total_yes24

,책제목,가격,출판사,출판일자
0,Do it! 점프 투 파이썬,"16,920",이지스퍼블리싱,2019년 06월
1,혼자 공부하는 파이썬,"19,800",한빛미디어,2022년 06월
2,만들면서 배우는 파이썬과 40개의 작품들,"16,920",앤써북,2022년 02월
3,혼자 공부하는 머신러닝+딥러닝,"23,400",한빛미디어,2020년 12월
4,파이썬 for Beginner,"24,250",한빛아카데미,2022년 01월
...,...,...,...,...
115,데이터 분석을 위한 파이썬 철저 입문,"25,200",위키북스,2019년 04월
116,파이썬으로 만드는 OpenCV 프로젝트,"32,400",인사이트(insight),2019년 03월
117,파이썬 데이터 분석,"28,000",생능출판사,2022년 02월
118,COS Pro 2급 파이썬 시험 대비서(교재+모의고사),"16,200",길벗,2018년 02월


In [161]:
# 엑셀로 저장
total_yes24.to_excel('yes24.xlsx')

# 2. 교보문고

In [135]:
# 2. 교보문고 크롤링
kyobo = 'http://www.kyobobook.co.kr/'
request = requests.get(kyobo)
print(request) # <Response [200]> 정상적인 접속

<Response [200]>


In [136]:
# chromedriver로 페이지 열기
driver = webdriver.Chrome('D:/tests/chromedriver.exe') 
driver.get(kyobo)
time.sleep(0.5)

c:\Users\user\AppData\Local\Programs\Python\Python37\lib\site-packages\ipykernel_launcher.py:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  


In [137]:
# 검색창에 파이썬 입력
search = driver.find_element('id', 'searchKeyword')
search.send_keys('파이썬')
search.send_keys(Keys.RETURN)
time.sleep(0.5)

In [143]:
# 책 정보 크롤링
total_kyobo = []
next = True
page = 1
index = 0
while next :
# https://search.kyobobook.co.kr/web/search?vPstrKeyWord=%25ED%258C%258C%25EC%259D%25B4%25EC%258D%25AC&vPstrTab=PRODUCT&searchPcondition=1&currentPage=2&orderClick=LAG#container

    # 2 ~ 5페이지로 이동시키고, 5페이지일 경우 멈추도록
    if page == 5 : 
        next = False
    if 2 <= page <= 5 :
        url = 'https://search.kyobobook.co.kr/web/search?vPstrKeyWord=%25ED%258C%258C%25EC%259D%25B4%25EC%258D%25AC&vPstrTab=PRODUCT&searchPcondition=1&currentPage={}&orderClick=LAG#container'.format(page)
        driver.get(url)
        driver.implicitly_wait(10)
        time.sleep(0.2)
    print("page : ", page)

    # bs4로 문서 정보를 가져옴
    html = driver.page_source
    soup = BeautifulSoup(html)

    # 페이지 도서 리스트
    book_list = soup.find('tbody', {'id' : 'search_list'})
    # 책 제목 태그 리스트
    book_name = book_list.select('div.title > a > strong')
    # 가격 태그 리스트
    book_price = book_list.select('div.sell_price > strong')
    # 카테고리 태그 리스트
    book_category = book_list.select('div.title > a > span.category2')


    print(len(book_name), len(book_price), len(book_category))
    # book_name list의 갯수만큼 반복문을 돌려 2차원 리스트의 형태로 저장
    for i in range(len(book_name)) :
        # 책 제목과 가격이 빈 값이 아닐 경우
        if book_name[i].text.strip() and book_price[i].text.strip() :
            # 책제목, 가격을 각각 저장
            books = [book_name[i].text.strip(), book_price[i].text.strip()[:-1], book_category[i].text.strip()]
            total_kyobo.append(books)
            index += 1
    page += 1
    time.sleep(0.2)


page :  1
20 20 20
page :  2
20 20 20
page :  3
20 20 20
page :  4
20 20 20
page :  5
20 20 20


In [144]:
# 이차원리스트를 데이터프레임으로 변환 및 컬럼 지정
total_kyobo_columns = ['책제목', '가격', '카테고리']
total_kyobo = pd.DataFrame(total_kyobo)
total_kyobo.columns = total_kyobo_columns
total_kyobo

,책제목,가격,카테고리
0,혼자 공부하는 파이썬,"19,800",[컴퓨터/IT]
1,Do it! 점프 투 파이썬,"16,920",[컴퓨터/IT]
2,만들면서 배우는 파이썬과 40개의 작품들,"16,920",[컴퓨터/IT]
3,파이썬 for Beginner,"24,250",[컴퓨터/IT]
4,비전공자를 위한 이해할 수 있는 IT 지식,"15,120",[컴퓨터/IT]
...,...,...,...
95,게임으로 배우는 파이썬,"15,300",[컴퓨터/IT]
96,초등 중등 코딩교육: 왕초보를 위한 PYTHON(파이썬) 기초 [1BD+1DVD],"33,000",[교양]
97,고성능 파이썬,"32,400",[컴퓨터/IT]
98,핵심만 쏙쏙 파이썬,"19,000",[컴퓨터/IT]


In [148]:
# 엑셀로 저장
total_kyobo.to_excel('kyobo.xlsx')

# 3. 알라딘

In [149]:
# 1. 알라딘 크롤링
aladin = 'https://www.aladin.co.kr/home/welcome.aspx'
request = requests.get(aladin) 
print(request) # <Response [200]> 정상적인 접속



<Response [200]>


In [150]:
# chromedriver로 페이지 열기
driver = webdriver.Chrome('D:/tests/chromedriver.exe') 
driver.get(aladin)
time.sleep(0.5)

c:\Users\user\AppData\Local\Programs\Python\Python37\lib\site-packages\ipykernel_launcher.py:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  


In [151]:
# 검색창에 파이썬 입력
search = driver.find_element('id', 'SearchWord')
search.send_keys('파이썬')
search.send_keys(Keys.RETURN)
time.sleep(0.5)

In [152]:
# 책 정보 크롤링
total_aladin = []
next = True
page = 1
index = 0
while next :
# https://www.aladin.co.kr/search/wsearchresult.aspx?SearchTarget=All&KeyWord=%ED%8C%8C%EC%9D%B4%EC%8D%AC&KeyRecentPublish=0&OutStock=0&ViewType=Detail&SortOrder=11&CustReviewCount=0&CustReviewRank=0&KeyFullWord=%ED%8C%8C%EC%9D%B4%EC%8D%AC&KeyLastWord=%ED%8C%8C%EC%9D%B4%EC%8D%AC&CategorySearch=&chkKeyTitle=&chkKeyAuthor=&chkKeyPublisher=&chkKeyISBN=&chkKeyTag=&chkKeyTOC=&chkKeySubject=&ViewRowCount=25&SuggestKeyWord=&page=2

    # 2 ~ 5페이지로 이동시키고, 5페이지일 경우 멈추도록
    if page == 5 : 
        next = False
    if 2 <= page <= 5 :
        url = 'https://www.aladin.co.kr/search/wsearchresult.aspx?SearchTarget=All&KeyWord=%ED%8C%8C%EC%9D%B4%EC%8D%AC&KeyRecentPublish=0&OutStock=0&ViewType=Detail&SortOrder=11&CustReviewCount=0&CustReviewRank=0&KeyFullWord=%ED%8C%8C%EC%9D%B4%EC%8D%AC&KeyLastWord=%ED%8C%8C%EC%9D%B4%EC%8D%AC&CategorySearch=&chkKeyTitle=&chkKeyAuthor=&chkKeyPublisher=&chkKeyISBN=&chkKeyTag=&chkKeyTOC=&chkKeySubject=&ViewRowCount=25&SuggestKeyWord=&page={}'.format(page)
        driver.get(url)
        driver.implicitly_wait(10)
        time.sleep(0.2)
    print("page : ", page)

    # bs4로 문서 정보를 가져옴
    html = driver.page_source
    soup = BeautifulSoup(html)

    # 페이지 도서 리스트
    book_list = soup.find('div', {'id' : 'Search3_Result'})
    # 책 제목 태그 리스트
    book_name = book_list.select('a.bo3 > b')
    # 가격 태그 리스트
    book_price = book_list.select('span.ss_p2 > b > span')
    # 세일즈포인트 태그 리스트
    book_point = book_list.select('ul>li:last-child > b')


    print(len(book_name), len(book_price), len(book_point))
    # book_name list의 갯수만큼 반복문을 돌려 2차원 리스트의 형태로 저장
    for i in range(len(book_name)) :
        # 책 제목과 가격이 빈 값이 아닐 경우
        if book_name[i].text.strip() and book_price[i].text.strip() :
            # 책제목, 가격, 세일즈포인트를 각각 저장
            books = [book_name[i].text.strip(), book_price[i].text.strip(), book_point[i].text.strip()]
            total_aladin.append(books)
            index += 1
    page += 1
    time.sleep(0.2)

page :  1
25 25 25
page :  2
25 25 25
page :  3
25 25 25
page :  4
25 25 25
page :  5
25 25 25


In [153]:
# 이차원리스트를 데이터프레임으로 변환 및 컬럼 지정
total_aladin_columns = ['책제목', '가격', '세일즈포인트']
total_aladin = pd.DataFrame(total_aladin)
total_aladin.columns = total_aladin_columns
total_aladin

,책제목,가격,세일즈포인트
0,Do it! 점프 투 파이썬,"16,920","29,692"
1,"클린 코드, 이제는 파이썬이다","27,000","1,900"
2,이것이 취업을 위한 코딩 테스트다 with 파이썬,"30,600","16,553"
3,Do it! 알고리즘 코딩 테스트 : 파이썬 편,"28,800","1,750"
4,혼자 공부하는 파이썬,"19,800","14,985"
...,...,...,...
120,파이썬 GUI 프로그래밍 쿡북 3/e,"31,500","1,091"
121,파이썬으로 경험하는 빅데이터 분석과 머신러닝,"27,000","1,202"
122,김변수와 시작하는 코딩생활 with 파이썬,"16,200","1,077"
123,컴퓨팅 사고와 파이썬,"24,000","1,069"


In [154]:
# 엑셀로 저장
total_aladin.to_excel('aladin.xlsx')